# DD142X

### Prerequisites
* Python v3.7
* Tensorflow v1.4.2 
* Installera massa skit med pip för att få det att fungera allteftersom Python klagar.
* Jag ändrade i train_image_classifier.py för att träna på CPU.
* Typ 100 GB hårddiskutrymme.

Det kan funka med andra versioner, men inte Tensorflow 2.

### Ladda in directories

In [21]:
# Set pretrained checkpoint dir.
%env PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained

# Where the training (fine-tuned) checkpoint and logs will be saved to.
%env TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints

# Where the testing results will be saved to.
%env EVAL_DIR=/Users/johan/Dev/kexet/eval

# Where the dataset is saved to.
%env DATASET_DIR=/Users/johan/Dev/kexet/data

env: PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained
env: TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints
env: EVAL_DIR=/Users/johan/Dev/kexet/eval
env: DATASET_DIR=/Users/johan/Dev/kexet/data


### Ladda ner en förtränad inception v3

Kör bara första gången.

In [ ]:
### Download the model if you haven't yet.
!mkdir ${PRETRAINED_CHECKPOINT_DIR}
!wget http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz
!tar -xvf inception_v3_2016_08_28.tar.gz
!mv inception_v3.ckpt ${PRETRAINED_CHECKPOINT_DIR}
!rm inception_v3_2016_08_28.tar.gz

### Konvertera ISIC data

Kör första gången och lägg sen manuellt in i mappar.

In [ ]:
import json
import os
import shutil

path_to_descriptions = '/Users/johan/Dev/kexet/rawdata'

final_path = '/Users/johan/Dev/kexet/data/skincancer'

for root, dirs, files in os.walk(path_to_descriptions):
    files = [f for f in files if not f[0] == '.']
    files = [f for f in files if not f[-1] == 'g']
    dirs[:] = [d for d in dirs if not d[0] == '.']
    
    for name in files:
        src = root + '/' + name
        try:
            with open(src) as data_file:
                #print(name)
                jsonList = json.load(data_file)

                disease = jsonList['meta']['clinical']['diagnosis']
                b_or_m = jsonList['meta']['clinical']['benign_malignant']
                if b_or_m is None:
                    b_or_m = 'neither'
                image_src_path = path_to_descriptions + '/Images/' + name + '.jpeg'
                disease_dir = final_path + '/' + disease + '/' + b_or_m
                if not os.path.exists(disease_dir):
                    os.makedirs(disease_dir)

                data_file.close()
                shutil.copy(image_src_path, disease_dir)
        except:
            continue
            

            
print("Finished!")

In [2]:
import download_and_convert_cancer as dcc
dcc.run("/Users/johan/Dev/kexet/data")


Training sample size n= 15223
Validation sample size n= 3806
Test sample size n= 2115



>> Converting images 1/15223 shard 0 file /Users/johan/Dev/kexet/data/nevus/ISIC_0029615.jpegWARNING:tensorflow:From /Users/johan/Dev/kexet/models/research/slim/download_and_convert_cancer.py:86: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

>> Converting images 15223/15223 shard 475 file /Users/johan/Dev/kexet/data/nevus/ISIC_0000847.jpegpeg0014576.jpeg
>> Converting images 3806/3806 shard 475 file /Users/johan/Dev/kexet/data/nevus/ISIC_0031576.jpegIC_0015006.jpeg
>> Converting images 2115/2115 shard 422 file /Users/johan/Dev/kexet/data/seborrheic_keratosis/ISIC_0014627.jpeg


Finished converting the Cancer dataset!


### Transfer learning
* Ladda vikterna från den generella färdigtränade modellen, men exkludera sista lagret med "checkpoint_exclude_scopes". 
* Träna om vikterna i detta lager med "trainable_scopes".

In [8]:
# Fine-tune only the new layers for 100 steps.
!python3 train_image_classifier.py \
  --train_dir=${TRAIN_DIR} \
  --dataset_name=cancer \
  --dataset_split_name=train \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3 \
  --checkpoint_path=${PRETRAINED_CHECKPOINT_DIR}/inception_v3.ckpt \
  --checkpoint_exclude_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --trainable_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --max_number_of_steps=100 \
  --batch_size=100 \
  --learning_rate=0.001 \
  --learning_rate_decay_type=fixed \
  --save_interval_secs=600 \
  --save_summaries_secs=600 \
  --log_every_n_steps=10 \
  --optimizer=rmsprop \
  --weight_decay=0.00004 \
  --opt_epsilon=0.1



W0512 15:59:49.564369 4649448896 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0512 15:59:49.564553 4649448896 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.create_global_step
W0512 15:59:49.565282 4649448896 deprecation.py:323] From train_image_classifier.py:431: create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step

W0512 15:59:49.569520 4649448896 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/cancer.py:37: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0512 15:59:49.570209 4649448896 module_w


W0512 15:59:49.705198 4649448896 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/preprocessing/inception_preprocessing.py:230: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
W0512 15:59:49.738528 4649448896 deprecation.py:323] From train_image_classifier.py:477: batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).

W0512 15:59:49.752135 4649448896 module_wrapper.py:139] From train_image_classifier.py:504: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.


W0512 15:59:49.752354 4649448896 modu

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0512 15:59:54.081400 4649448896 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

W0512 15:59:54.167237 4649448896 module_wrapper.py:139] From train_image_classifier.py:588: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.


W0512 15:59:54.179909 4649448896 module_wrapper.py:139] From train_image_classifier.py:363: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Ignoring --checkpoint_path because a checkpoint already exists in /Users/johan/Dev/kexet/checkpoints
I0512 15:59:54

In [40]:


# Run evaluation.
!python3 eval_image_classifier.py \
  --checkpoint_path=${TRAIN_DIR} \
  --eval_dir=${EVAL_DIR} \
  --dataset_name=cancer \
  --dataset_split_name=test \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3



W0512 16:12:58.209841 4630582720 module_wrapper.py:139] From eval_image_classifier.py:97: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0512 16:12:58.210072 4630582720 module_wrapper.py:139] From eval_image_classifier.py:97: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0512 16:12:58.210855 4630582720 deprecation.py:323] From eval_image_classifier.py:99: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Traceback (most recent call last):
  File "eval_image_classifier.py", line 203, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.7/site-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv

In [39]:
!cd ${TRAIN_DIR}
!grep -E "model\.(.+)\.data" ${TRAIN_DIR}

!pwd
#%env CHECKPOINT_PATHS = ls | grep -E "model\.(.+)\.data") 
!ls | grep -E "model\.(.+)\.data"

#test = ${CHECKPOINT_PATHS}
#print(test)

grep: /Users/johan/Dev/kexet/checkpoints: Is a directory
/Users/johan/Dev/kexet/models/research/slim
/bin/sh: ls : command not found


In [18]:
!ls

BUILD                          eval_image_classifier.py
DD142X.ipynb                   export_inference_graph.py
README.md                      export_inference_graph_test.py
WORKSPACE                      nets
__init__.py                    preprocessing
__pycache__                    scripts
datasets                       setup.py
deployment                     slim_walkthrough.ipynb
download_and_convert_cancer.py train_image_classifier.py
download_and_convert_data.py
